# Utils

**Импорты библиотек:**

In [1]:
!pip install -q tqdm==4.66.5
!pip install -q torch==2.4.1+cu121
!pip install -q torchvision==0.19.1+cu121
!pip install -q transformers==4.44.2
!pip install -q comet_ml==3.47.6
!pip install -q triton==3.1.0
!pip install -q torchattacks==3.5.1
!pip install -q jax==0.4.38
!pip install -q jaxlib==0.4.38
!pip install -q seaborn


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement torch==2.4.1+cu121 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1)
ERROR: No matching distribution found for torch==2.4.1+cu121

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
ERROR: Ignored the following yanked versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.15.0
ERROR: Could not find a version that satisfies the requirement torchvision==0.19.1+cu121 (from versions: 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.16.2, 0.17.0, 0.17.1, 0.17.2, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 0.20.0, 0.20.1)
ERROR: No matching distribution found for torchvision==0.19.1+cu121

[notice

In [2]:
import random

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import os

import comet_ml

import torch
import torchvision
from torchvision.transforms import v2
import torchattacks
from transformers import AutoImageProcessor, ViTMAEForPreTraining, ViTMAEConfig

**Utils:**

**Функция для фиксирования сида:**

In [3]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

**Additional utils:**

In [4]:
def get_root(env):
    # supports env == "colab", env == "kaggle"
    if env == "kaggle":
        return "/kaggle/working/"
    else:
        return "./"

In [5]:
# this augmentation first converts PIL image to Tensor,
# then casts resulting tensor to type=torch.float32,
# finally normalizes the image

default_aug = v2.Compose([v2.PILToTensor(),
                          v2.ToDtype(torch.float32, scale=True),
                          v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

In [6]:
# This function is used later in DataLoaders to fix the seed setting bugs when using several workers

def worker_init_fn(worker_id):
    torch_seed = torch.initial_seed() % (2**32)
    random.seed(torch_seed)
    np.random.seed(torch_seed)

**Датасет + DataLoaders:**

In [7]:
def get_CIFAR10_data(train_transform=default_aug, test_transform=default_aug, env="colab"):
    root = get_root(env)
    data_train = torchvision.datasets.CIFAR10(root=root, train=True, transform=train_transform, download=True)
    print("Prepare Train Set: ✅")
    data_test = torchvision.datasets.CIFAR10(root=root, train=False, transform=test_transform)
    print("Prepare Test Set:  ✅")
    return data_train, data_test

In [8]:
def get_CIFAR10_dataloaders(batch_size, train_transform=default_aug, test_transform=default_aug, env="colab"):
    root = get_root(env)
    data_train = torchvision.datasets.CIFAR10(root=root, train=True, transform=train_transform, download=True)
    dataloader_train = torch.utils.data.DataLoader(data_train, batch_size=batch_size, shuffle=True, num_workers=4, worker_init_fn=worker_init_fn)
    print("Prepare Train Set: ✅")
    data_test = torchvision.datasets.CIFAR10(root=root, train=False, transform=test_transform)
    dataloader_test = torch.utils.data.DataLoader(data_test, batch_size=batch_size, shuffle=False, num_workers=4, worker_init_fn=worker_init_fn)
    print("Prepare Test Set:  ✅")
    return dataloader_train, dataloader_test

**Код цикла предобучения/дообучения SSL-метода:**

In [9]:
def pretrain_epoch(model, optimizer, loader, scaler, epoch, exp):
    step = (epoch - 1) * len(loader)

    model.train()

    for batch_idx, (data, _) in enumerate(tqdm(loader)):
        data = data.to(memory_format=torch.channels_last) # this line is for a speedup (used in a combination with amp)
        data = data.to(device)

        optimizer.zero_grad()

        with torch.autocast("cuda", dtype=torch.float16, enabled=True):
            loss = model(data).loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if exp is not None:
            exp.log_metrics({"SSL train loss (by steps)": loss.item()}, step=(step + batch_idx))

    return scaler


def pretrain(model, exp_name, optimizer, epochs, pretrain_loader, scheduler=None, exp=None, env="colab"):
    scaler = torch.GradScaler("cuda")
    
    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}:\n")
        scaler = pretrain_epoch(model, optimizer, pretrain_loader, scaler, epoch, exp)
        
        if scheduler is not None:
            scheduler.step()

    filepath = f"{get_root(env)}weights_{exp_name}_epoch{epochs}.pth"
    torch.save(model.state_dict(), filepath)
    print("Save State of the Best Model: ✅")

    if exp is not None:
        exp.log_model(name=filepath[len(get_root(env)):-4],
                      file_or_folder=filepath)

In [10]:
comet_ml.login()

Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)


Comet API key:  ········


COMET INFO: Valid Comet API Key saved in /home/mlcore/.comet.config (set COMET_CONFIG to change where it is saved).


In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
NUM_EPOCHS = 50
BATCH_SIZE = 128
exp_name = "MAE_Pretrain"

In [13]:
# random seed
seed_everything(0)

#model
mae_config = ViTMAEConfig.from_pretrained("facebook/vit-mae-base")
mae_config.image_size = 32
mae_config.patch_size = 8

model = ViTMAEForPreTraining(mae_config).to(memory_format=torch.channels_last)
model.to(device)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="colab")

# choose optimizer and scheduler (if needed)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=1e-5)

config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

Files already downloaded and verified
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [14]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial1")
exp.set_name(exp_name)

pretrain(model, exp_name, optimizer, NUM_EPOCHS, train_loader, scheduler=scheduler, exp=exp, env="colab")

exp.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/askoro/ssl-adversarial1/4076316c21924aa99818ca7e97539c57



Epoch 1:



COMET INFO: Couldn't find a Git repository in '/workdir' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
100%|██████████| 391/391 [00:16<00:00, 23.80it/s]


Epoch 2:



100%|██████████| 391/391 [00:15<00:00, 25.21it/s]


...

100%|██████████| 391/391 [00:15<00:00, 24.64it/s]


Epoch 50:



100%|██████████| 391/391 [00:15<00:00, 25.14it/s]


Save State of the Best Model: ✅


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : MAE_Pretrain
COMET INFO:     url                   : https://www.comet.com/askoro/ssl-adversarial1/4076316c21924aa99818ca7e97539c57
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     SSL train loss (by steps) [19550] : (0.09143493324518204, 0.4419786036014557)
COMET INFO:     loss [1955]                       : (8002.61572265625, 2210196.75)
COMET INFO:   Others:
COMET INFO:     Name : MAE_Pretrain
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     model-element       : 1 (423.34 MB)
COMET INFO:     notebook            : 1
COMET INFO:   

**Код для обычного/adversarial Fine-Tuning'а (на одном GPU):**

In [15]:
def eval_epoch(model, loader, attack=None):
    val_loss = 0.
    val_acc = 0.

    model.eval()

    for data, target in tqdm(loader):
        data = data.to(memory_format=torch.channels_last) # this line is for a speedup (used in a combination with amp)
        data = data.to(device)
        target = target.to(device)

        if attack is not None:
            # perform adversarial attack
            with torch.enable_grad():
                data = attack(data, target)
        
        logits = model(data)
        loss = torch.nn.functional.cross_entropy(logits, target)

        val_loss += (loss.item() * data.shape[0])
        val_acc += (torch.argmax(logits, dim=1) == target).sum().item()

    val_loss /= len(loader.dataset)
    val_acc /= len(loader.dataset)
    return val_loss, val_acc


def train_epoch(model, optimizer, loader, scaler, epoch, exp, is_attack_used=False):
    train_loss = 0.
    train_acc = 0.

    step = (epoch - 1) * len(loader)

    model.train()

    for batch_idx, (data, target) in enumerate(tqdm(loader)):
        data = data.to(memory_format=torch.channels_last) # this line is for a speedup (used in a combination with amp)
        data = data.to(device)
        target = target.to(device)

        if is_attack_used != "no":
            if is_attack_used == "BIM":
                attack = torchattacks.BIM(model, eps=(4 / 255), alpha=(2 / 255), steps=5)
            else:
                attack = torchattacks.PGD(model, eps=(4 / 255), alpha=(2 / 255), steps=5)
            # perform adversarial attack
            data = attack(data, target)

        optimizer.zero_grad()

        with torch.autocast("cuda", dtype=torch.float16, enabled=True):
            logits = model(data)
            loss = torch.nn.functional.cross_entropy(logits, target)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        with torch.no_grad():
            train_loss += loss.item() * data.shape[0]
            train_acc += (torch.argmax(logits, dim=1) == target).sum().item()

        if exp is not None:
            exp.log_metrics({"train loss (by steps)": loss.item()}, step=(step + batch_idx))

    train_loss /= len(loader.dataset)
    train_acc /= len(loader.dataset)
    return train_loss, train_acc, scaler


def train(model, exp_name, optimizer, epochs, train_loader, val_loader, is_train_attack_used="no", scheduler=None, exp=None, env="colab"):
    best_val_acc = 0.
    best_val_acc_epoch = None
    best_model_state_filepath = None

    val_attacks_names = ["No Attack"]

    scaler = torch.GradScaler("cuda")

    for epoch in range(1, epochs + 1):
        print(f"Epoch {epoch}:\n")
        metrics = dict()
        
        train_loss, train_acc, scaler = train_epoch(model, optimizer, train_loader, scaler, epoch, exp, is_attack_used=is_train_attack_used)

        val_attacks = [None]
        
        for it in range(len(val_attacks)):
            val_loss, val_acc = eval_epoch(model, val_loader, attack=val_attacks[it])
            metrics[val_attacks_names[it]] = {"Loss": val_loss, "Accuracy": val_acc}

        if scheduler is not None:
            scheduler.step()

        print("\n")
        print(" Train:")
        print(f"  Loss: {train_loss}, Accuracy: {train_acc}")
        print(" Validation:")
        for it in metrics.items():
            print(f"  Loss: {it[1]['Loss']}, Accuracy: {it[1]['Accuracy']}")


        if exp is not None:
            exp.log_metrics(
                   {"train loss": train_loss,
                    "train accuracy": train_acc},
                    step=(epoch * len(train_loader)))
            for it in metrics.items():
                exp.log_metrics(
                    {f"validation loss ({it[0]})": it[1]['Loss'],
                     f"validation accuracy ({it[0]})": it[1]['Accuracy']},
                     step=(epoch * len(train_loader)))

        val_acc = metrics["No Attack"]["Accuracy"]
        
        if val_acc > best_val_acc:
            print("🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉")
            best_val_acc = val_acc
            best_val_acc_epoch = epoch
            filepath = f"{get_root(env)}best_weights_{exp_name}_epoch{best_val_acc_epoch}.pth"
            torch.save(model.state_dict(), filepath)
            print("Save State of the Best Model: ✅")
            if best_model_state_filepath is not None:
                os.remove(best_model_state_filepath)
                print("Delete State File of the Previous Best Model: ✅")
            best_model_state_filepath = filepath

        print('-----------------------------------------------------------------\n')

    if exp is not None:
        exp.log_model(name=best_model_state_filepath[len(get_root(env)):-4],
                      file_or_folder=best_model_state_filepath)


def test(model, loader, env="colab"):
    attacks_names = ["No Attack", "i-FGSM", "PGD", "DI-FGSM"]
    
    metrics = dict()

    attacks = [None, 
               torchattacks.BIM(model, eps=(8 / 255), alpha=(2 / 255), steps=10),
               torchattacks.PGD(model, eps=(8 / 255), alpha=(2 / 255), steps=10),
               torchattacks.DIFGSM(model, eps=(8 / 255), alpha=(2 / 255), steps=10, resize_rate=0.9, diversity_prob=0.5)
              ]
    
    for it in range(len(attacks)):
        loss, acc = eval_epoch(model, loader, attack=attacks[it])
        metrics[attacks_names[it]] = {"Loss": loss, "Accuracy": acc}

    print("\n")
    print("Final Metrics:")
    for it in metrics.items():
        print(f" {it[0]}:")
        print(f"   Loss: {it[1]['Loss']}, Accuracy: {it[1]['Accuracy']}")


# Эксперименты

**Downstream-модель:**

In [16]:
class DownstreamModel(torch.nn.Module):
    def __init__(self, encoder, head):
        super(DownstreamModel, self).__init__()
        self.encoder = encoder
        self.head = head

    def forward(self, x):
        enc_features = self.encoder(x).last_hidden_state[:, 0, :]
        logits = self.head(enc_features)
        return logits

**Берём энкодер из модели, прикрепляем линейную голову:**

In [17]:
seed_everything(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mae_config = ViTMAEConfig.from_pretrained("facebook/vit-mae-base")
mae_config.image_size = 32
mae_config.patch_size = 8

model = ViTMAEForPreTraining(mae_config).to(memory_format=torch.channels_last)
model.to(device)

model_state_dict = torch.load("./weights_MAE_Pretrain_epoch50.pth", map_location=device, weights_only=False)

model.load_state_dict(model_state_dict)

# here I take only the backbone of the model and then use it as an encoder in a downstream model 
encoder = model.vit

# initialize new head
NUM_CLASSES = 10 # there are 10 classes in CIFAR10
HIDDEN_SIZE = 768 # hidden size of MAE

head = torch.nn.Sequential(
    torch.nn.Linear(HIDDEN_SIZE, 2 * HIDDEN_SIZE),
    torch.nn.GELU(),
    torch.nn.Linear(2 * HIDDEN_SIZE, NUM_CLASSES)
)

# create downstream model
model = DownstreamModel(encoder=encoder, head=head).to(memory_format=torch.channels_last)
model.to(device)

DownstreamModel(
  (encoder): ViTMAEModel(
    (embeddings): ViTMAEEmbeddings(
      (patch_embeddings): ViTMAEPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(8, 8), stride=(8, 8))
      )
    )
    (encoder): ViTMAEEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTMAELayer(
          (attention): ViTMAESdpaAttention(
            (attention): ViTMAESdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTMAESelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTMAEIntermediate(
            (dense): Linear(in_features=768, out_features=3072,

**MAE (SFT):**

In [18]:
comet_ml.login()

In [19]:
# choose device for computing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
NUM_EPOCHS = 10
BATCH_SIZE = 128
exp_name = "MAE_SFT"

In [21]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="colab")

# choose optimizer and scheduler (if needed)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

Files already downloaded and verified
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [22]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial1")
exp.set_name(exp_name)

train(model, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="no", scheduler=scheduler, exp=exp, env="colab")

exp.end()

print()
test(model, test_loader, env="colab")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/askoro/ssl-adversarial1/5568368033474f3fae3bd9b1783a9e21

COMET INFO: Couldn't find a Git repository in '/workdir' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Epoch 1:



100%|██████████| 79/79 [00:01<00:00, 52.01it/s]




 Train:
  Loss: 1.6217463385391235, Accuracy: 0.4109
 Validation:
  Loss: 1.532886388015747, Accuracy: 0.4478
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:01<00:00, 53.09it/s]


...

100%|██████████| 79/79 [00:01<00:00, 56.19it/s]




 Train:
  Loss: 1.2454099044799805, Accuracy: 0.55164
 Validation:
  Loss: 1.3267727630615234, Accuracy: 0.5236
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
Delete State File of the Previous Best Model: ✅
-----------------------------------------------------------------



COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : MAE_SFT
COMET INFO:     url                   : https://www.comet.com/askoro/ssl-adversarial1/5568368033474f3fae3bd9b1783a9e21
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [392]                           : (79577.03125, 200933.828125)
COMET INFO:     train accuracy [10]                  : (0.4109, 0.55164)
COMET INFO:     train loss (by steps) [3910]         : (0.9872626066207886, 2.3300628662109375)
COMET INFO:     train loss [10]                      : (1.2454099044799805, 1.6217463385391235)
COMET INFO:     validation accuracy (No Attack) [10] : (0.4478, 0.5236)
COMET INFO:     validation loss (No Attack) [10]     : (1.3267727630

100%|██████████| 79/79 [00:10<00:00,  7.83it/s]



Final Metrics:
 No Attack:
   Loss: 1.3230202751159668, Accuracy: 0.525
 i-FGSM:
   Loss: 1.578238966178894, Accuracy: 0.4379
 PGD:
   Loss: 2.553281261062622, Accuracy: 0.1965
 DI-FGSM:
   Loss: 2.497017435836792, Accuracy: 0.2161


**MAE (Adversarial SFT with i-FGSM):**

In [23]:
seed_everything(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mae_config = ViTMAEConfig.from_pretrained("facebook/vit-mae-base")
mae_config.image_size = 32
mae_config.patch_size = 8

model = ViTMAEForPreTraining(mae_config).to(memory_format=torch.channels_last)
model.to(device)

model_state_dict = torch.load("./weights_MAE_Pretrain_epoch50.pth", map_location=device, weights_only=False)

model.load_state_dict(model_state_dict)

# here I take only the backbone of the model and then use it as an encoder in a downstream model 
encoder = model.vit

# initialize new head
NUM_CLASSES = 10 # there are 10 classes in CIFAR10
HIDDEN_SIZE = 768 # hidden size of MAE

head = torch.nn.Sequential(
    torch.nn.Linear(HIDDEN_SIZE, 2 * HIDDEN_SIZE),
    torch.nn.GELU(),
    torch.nn.Linear(2 * HIDDEN_SIZE, NUM_CLASSES)
)

# create downstream model
model = DownstreamModel(encoder=encoder, head=head).to(memory_format=torch.channels_last)
model.to(device)

DownstreamModel(
  (encoder): ViTMAEModel(
    (embeddings): ViTMAEEmbeddings(
      (patch_embeddings): ViTMAEPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(8, 8), stride=(8, 8))
      )
    )
    (encoder): ViTMAEEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTMAELayer(
          (attention): ViTMAESdpaAttention(
            (attention): ViTMAESdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTMAESelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTMAEIntermediate(
            (dense): Linear(in_features=768, out_features=3072,

In [24]:
comet_ml.login()

In [25]:
# choose device for computing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
NUM_EPOCHS = 10
BATCH_SIZE = 128
exp_name = "MAE_Adversarial_SFT_BIM"

In [27]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="colab")

# choose optimizer and scheduler (if needed)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

Files already downloaded and verified
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [28]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial1")
exp.set_name(exp_name)

train(model, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="BIM", scheduler=scheduler, exp=exp, env="colab")

exp.end()

print()
test(model, test_loader, env="colab")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/askoro/ssl-adversarial1/e292173924df4c3f9f1eadf18c9f9102

COMET INFO: Couldn't find a Git repository in '/workdir' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Epoch 1:



100%|██████████| 79/79 [00:01<00:00, 54.45it/s]




 Train:
  Loss: 1.6762386400604248, Accuracy: 0.38708
 Validation:
  Loss: 1.5449791828155517, Accuracy: 0.4432
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:01<00:00, 53.40it/s]


...

100%|██████████| 79/79 [00:01<00:00, 55.89it/s]




 Train:
  Loss: 1.324295258102417, Accuracy: 0.5202
 Validation:
  Loss: 1.3419679927825927, Accuracy: 0.5157
-----------------------------------------------------------------



COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : MAE_Adversarial_SFT_BIM
COMET INFO:     url                   : https://www.comet.com/askoro/ssl-adversarial1/e292173924df4c3f9f1eadf18c9f9102
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [392]                           : (84278.515625, 210467.96875)
COMET INFO:     train accuracy [10]                  : (0.38708, 0.52066)
COMET INFO:     train loss (by steps) [3910]         : (1.0689847469329834, 2.3407745361328125)
COMET INFO:     train loss [10]                      : (1.324295258102417, 1.6762386400604248)
COMET INFO:     validation accuracy (No Attack) [10] : (0.4432, 0.5162)
COMET INFO:     validation loss (No Attack) [10]    

100%|██████████| 79/79 [00:10<00:00,  7.81it/s]



Final Metrics:
 No Attack:
   Loss: 1.3359559963226317, Accuracy: 0.5199
 i-FGSM:
   Loss: 1.4854160963058471, Accuracy: 0.4658
 PGD:
   Loss: 2.3496190362930296, Accuracy: 0.2346
 DI-FGSM:
   Loss: 2.34633367729187, Accuracy: 0.2319


In [29]:
test(model, test_loader, env="colab")

100%|██████████| 79/79 [00:10<00:00,  7.77it/s]



Final Metrics:
 No Attack:
   Loss: 1.3388384386062622, Accuracy: 0.519
 i-FGSM:
   Loss: 1.475374172782898, Accuracy: 0.4713
 PGD:
   Loss: 2.3680355110168456, Accuracy: 0.2268
 DI-FGSM:
   Loss: 2.3492963069915773, Accuracy: 0.2371


**MAE (Adversarial SFT with PGD):**

In [30]:
seed_everything(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

mae_config = ViTMAEConfig.from_pretrained("facebook/vit-mae-base")
mae_config.image_size = 32
mae_config.patch_size = 8

model = ViTMAEForPreTraining(mae_config).to(memory_format=torch.channels_last)
model.to(device)

model_state_dict = torch.load("./weights_MAE_Pretrain_epoch50.pth", map_location=device, weights_only=False)

model.load_state_dict(model_state_dict)

# here I take only the backbone of the model and then use it as an encoder in a downstream model 
encoder = model.vit

# initialize new head
NUM_CLASSES = 10 # there are 10 classes in CIFAR10
HIDDEN_SIZE = 768 # hidden size of MAE

head = torch.nn.Sequential(
    torch.nn.Linear(HIDDEN_SIZE, 2 * HIDDEN_SIZE),
    torch.nn.GELU(),
    torch.nn.Linear(2 * HIDDEN_SIZE, NUM_CLASSES)
)

# create downstream model
model = DownstreamModel(encoder=encoder, head=head).to(memory_format=torch.channels_last)
model.to(device)

DownstreamModel(
  (encoder): ViTMAEModel(
    (embeddings): ViTMAEEmbeddings(
      (patch_embeddings): ViTMAEPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(8, 8), stride=(8, 8))
      )
    )
    (encoder): ViTMAEEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTMAELayer(
          (attention): ViTMAESdpaAttention(
            (attention): ViTMAESdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTMAESelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTMAEIntermediate(
            (dense): Linear(in_features=768, out_features=3072,

In [31]:
comet_ml.login()

In [32]:
# choose device for computing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [33]:
NUM_EPOCHS = 10
BATCH_SIZE = 128
exp_name = "MAE_Adversarial_SFT_PGD"

In [34]:
# random seed
seed_everything(0)

# get data and batchify it
train_loader, test_loader = get_CIFAR10_dataloaders(batch_size=BATCH_SIZE, env="colab")

# choose optimizer and scheduler (if needed)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)

Files already downloaded and verified
Prepare Train Set: ✅
Prepare Test Set:  ✅


In [35]:
exp = comet_ml.Experiment(project_name="SSL_Adversarial1")
exp.set_name(exp_name)

train(model, exp_name, optimizer, NUM_EPOCHS, train_loader, test_loader, is_train_attack_used="PGD", scheduler=scheduler, exp=exp, env="colab")

exp.end()

print()
test(model, test_loader, env="colab")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/askoro/ssl-adversarial1/ec5e0ff1868546649864368bb483eb77

COMET INFO: Couldn't find a Git repository in '/workdir' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


Epoch 1:



100%|██████████| 79/79 [00:01<00:00, 54.63it/s]




 Train:
  Loss: 1.8927465064239501, Accuracy: 0.3063
 Validation:
  Loss: 1.7694414241790772, Accuracy: 0.3567
🎉🎉🎉 New Best Validation Accuracy 🎉🎉🎉
Save State of the Best Model: ✅
-----------------------------------------------------------------

Epoch 2:



100%|██████████| 79/79 [00:01<00:00, 54.81it/s]


...

100%|██████████| 79/79 [00:01<00:00, 54.91it/s]




 Train:
  Loss: 1.611339677734375, Accuracy: 0.4185
 Validation:
  Loss: 1.9045633159637452, Accuracy: 0.3031
-----------------------------------------------------------------



COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : MAE_Adversarial_SFT_PGD
COMET INFO:     url                   : https://www.comet.com/askoro/ssl-adversarial1/ec5e0ff1868546649864368bb483eb77
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [392]                           : (98887.84375, 243138.6875)
COMET INFO:     train accuracy [10]                  : (0.3063, 0.4185)
COMET INFO:     train loss (by steps) [3910]         : (1.3626270294189453, 2.3716888427734375)
COMET INFO:     train loss [10]                      : (1.611339677734375, 1.8927465064239501)
COMET INFO:     validation accuracy (No Attack) [10] : (0.3031, 0.3567)
COMET INFO:     validation loss (No Attack) [10]     : (

100%|██████████| 79/79 [00:10<00:00,  7.74it/s]



Final Metrics:
 No Attack:
   Loss: 1.8994468881607056, Accuracy: 0.3052
 i-FGSM:
   Loss: 2.0006841358184815, Accuracy: 0.2757
 PGD:
   Loss: 1.7425093286514282, Accuracy: 0.3673
 DI-FGSM:
   Loss: 1.7365117275238038, Accuracy: 0.3769
